In [2]:
from pyrosm import OSM
from pyrosm import get_data

import geopandas as gpd
import pandas as pd

In [3]:
jalan=gpd.read_file('jalan_nusra.gpkg')

In [4]:
jalan.head()

,access,area,bicycle,bicycle_road,bridge,cycleway,est_width,foot,footway,highway,...,tracktype,tunnel,width,id,timestamp,version,tags,osm_type,length,geometry
0,None,None,None,None,None,None,None,None,None,secondary,...,None,None,None,23367072,1524637690,9,None,way,250.0,"MULTILINESTRING ((115.22755 -8.79635, 115.2275..."
1,None,None,None,None,None,None,None,None,None,tertiary,...,None,None,None,23367100,1638778902,32,"{""postal_code"":""80362""}",way,895.0,"MULTILINESTRING ((115.18774 -8.78596, 115.1877..."
2,None,None,None,None,None,None,None,None,None,residential,...,None,None,None,23367103,1568446502,12,None,way,220.0,"MULTILINESTRING ((115.19325 -8.79023, 115.1932..."
3,None,None,None,None,None,None,None,None,None,residential,...,None,None,None,23367106,1586758349,15,None,way,188.0,"MULTILINESTRING ((115.19317 -8.78208, 115.1931..."
4,None,None,None,None,None,None,None,None,None,residential,...,None,None,None,23367107,1637220100,12,None,way,340.0,"MULTILINESTRING ((115.19354 -8.78220, 115.1935..."


In [5]:
highway_l=['secondary', 'tertiary', 'primary',
       'trunk','motorway' ]
surface_l=['paved', 'asphalt', 'unpaved', None,'cobblestone', 
       'paving_stones', 'gravel;asphalt', 'concrete', 'asphalt_ruts',
       'Hotmix', 'dirt;asphalt', 'metal',
       'sett', 'concrete:plates',
       'compacted', 'Aspal dan Tanah',
       'permukaan tanah dan Beton', 'Tanah dan Aspal', 'Aspal dan Beton', 'Tanah dan Beton',
       'concrete:lanes', 'Rabat', 'Hotmix dan Tanah',
       'paving_stones:30', 'driveway', 'unhewn_cobblestone']
jalan.query('highway in @highway_l').query('surface in @surface_l').to_file('data_jalan_nusra.gpkg',driver='GPKG')

C:\Users\User\Anaconda3\envs\pysal_env\lib\site-packages\geopandas\io\file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,


In [2]:
# Initialize the OSM parser object
osm = OSM("nusa-tenggara-latest-internal.osm.pbf")

In [3]:
drive_net = osm.get_network(network_type="driving")

In [4]:
drive_net.surface.unique()

array(['paved', 'asphalt', 'unpaved', 'cobblestone', None,
       'paving_stones', 'gravel;asphalt', 'concrete', 'asphalt_ruts',
       'ground', 'Hotmix', 'dirt', 'dirt;asphalt', 'gravel', 'metal',
       'grass', 'mud', 'sand', 'sett', 'concrete:plates', 'fine_gravel',
       'compacted', 'Aspal dan Tanah', 'pebblestone',
       'permukaan tanah dan Beton', 'grass_paver', 'Tanah dan Aspal',
       'Tanah', 'Aspal dan Beton', 'Tanah dan Beton',
       'permukaan tanah dan Berbatu', 'TANAH KANCING', 'concrete:lanes',
       'wood', 'Trotoar', 'Rabat', 'PAVING BLOK', 'Hotmix dan Tanah',
       'paving_stones:30', 'driveway', 'Travessa_De_Ai-Kakau', 'clay',
       'dirt/sand', 'unpaved_pathway', 'בלתי_סלול', 'unhewn_cobblestone',
       'com', 'groun'], dtype=object)

In [5]:
drive_net.highway.unique()

array(['secondary', 'tertiary', 'residential', 'primary', 'service',
       'living_street', 'unclassified', 'trunk', 'trunk_link',
       'pedestrian', 'path', 'secondary_link', 'primary_link',
       'tertiary_link', 'emergency_access_point', 'track', 'footway',
       'rest_area', 'motorway', 'motorway_link', 'road', 'cycleway'],
      dtype=object)

In [6]:
drive_net.to_file('data_jalan_osm.gpkg',driver='GPKG')

In [9]:
gf=gpd.read_file('degurba_5300_rev_w_buitup_adjustment_3.gpkg')

In [10]:
highway_=drive_net.query('highway in @highway_l').query('surface in @surface_l')

In [11]:
data_highway=highway_.overlay(gf.dissolve(),how='intersection')

In [12]:
high_2km=data_highway.to_crs(3857).buffer(2000).to_crs(4326)

In [13]:
data_highway['geometry']=high_2km

In [14]:
data_jalan=data_highway[['geometry']]
data_jalan['reg']=1

C:\Users\user\.conda\envs\pysal_env\lib\site-packages\geopandas\geodataframe.py:1456: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [15]:
data_jalan.to_file('jalan_2km_53.gpkg',driver='GPKG')

In [16]:
data_int=gpd.sjoin(gf,data_jalan,how='left',predicate='intersects')

In [17]:
data_int.reg.unique()

array([ 1., nan])

In [18]:
unique_nama=data_int.query('reg==1')['Name'].unique()

In [19]:
gf['accessed']=gf.Name.apply(lambda y:1 if y in unique_nama else 0)

In [20]:
data_p=gf.pivot_table(index='classification',columns='accessed',values='pop_built_s2',aggfunc='sum').fillna(0)

In [21]:
data_p['percent']=data_p[1]*100/(data_p[0]+data_p[1])

In [22]:
data_p

accessed,0,1,percent
classification,,,
CENTRE,1587.229541,6.706639e+05,99.763893
CLUSTER,142988.889603,1.429182e+06,90.905004
RURAL,422199.444361,1.395673e+06,76.775084


In [23]:
gf.to_file('rai_degurba_ntt.gpkg',driver='GPKG')